In [3]:
import json

faq_data = [
  {"question": "What is AI?", "answer": "AI is the simulation of human intelligence in machines."},
  {"question": "What is Machine Learning?", "answer": "Machine learning is a subset of AI where systems learn from data."},
  {"question": "What is Deep Learning?", "answer": "Deep learning is a subset of ML using neural networks with many layers."},
  {"question": "What is Natural Language Processing?", "answer": "NLP is a field of AI focused on interaction between computers and human language."},
  {"question": "What is Computer Vision?", "answer": "Computer vision enables machines to interpret and process visual information."},
  {"question": "What is Reinforcement Learning?", "answer": "Reinforcement learning is a type of ML where agents learn through trial and error."},
  {"question": "What is supervised learning?", "answer": "Supervised learning is training a model on labeled data."},
  {"question": "What is unsupervised learning?", "answer": "Unsupervised learning finds patterns in unlabeled data."},
  {"question": "What is a neural network?", "answer": "A neural network is a model inspired by the human brain, used to recognize patterns."},
  {"question": "What is the difference between AI and ML?", "answer": "AI is the broader concept, while ML is a subset of AI."}
]

with open("faq.json", "w") as f:
    json.dump(faq_data, f, indent=2)

print("faq.json ✅")



faq.json ✅


# Report: In-Context Learning for FAQ Bot

In [4]:
!curl -fsSL https://ollama.com/install.sh | sh
!ollama pull mistral


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it


In [5]:
import json

with open("faq.json", "r") as f:
    faq_data = json.load(f)

faq_data[:3]  # show first 3 entries


[{'question': 'What is AI?',
  'answer': 'AI is the simulation of human intelligence in machines.'},
 {'question': 'What is Machine Learning?',
  'answer': 'Machine learning is a subset of AI where systems learn from data.'},
 {'question': 'What is Deep Learning?',
  'answer': 'Deep learning is a subset of ML using neural networks with many layers.'}]

In [6]:
def build_context(faq_data, user_query):
    context = "You are an AI FAQ assistant. Use the dataset below to answer questions.\n\n"
    for item in faq_data:
        context += f"Q: {item['question']}\nA: {item['answer']}\n\n"
    context += f"User: {user_query}\nAssistant:"
    return context


In [7]:
import subprocess
import json

def ask_ollama(user_query, faq_data):
    prompt = build_context(faq_data, user_query)

    # Run Ollama with subprocess
    result = subprocess.run(
        ["ollama", "run", "mistral"],
        input=prompt.encode("utf-8"),
        capture_output=True,
    )
    return result.stdout.decode("utf-8").strip()


In [8]:
def faq_bot(user_query, faq_data):
    # simple match check
    for item in faq_data:
        if user_query.lower() in item['question'].lower():
            return item['answer']

    # otherwise, use Ollama to guess
    response = ask_ollama(user_query, faq_data)

    if response.strip() == "" or "I don’t know" in response:
        return "I don’t know, please refine your question."
    return response


In [9]:
while True:
    user_query = input("Ask a question (or type 'exit' to quit): ")
    if user_query.lower() == "exit":
        break
    print("Bot:", faq_bot(user_query, faq_data))


Ask a question (or type 'exit' to quit): What is Reinforcement learning
Bot: Reinforcement learning is a type of ML where agents learn through trial and error.
Ask a question (or type 'exit' to quit): Explain details
Bot: I don’t know, please refine your question.
Ask a question (or type 'exit' to quit): exit


#### How In-Context Learning was Applied
In this project, **In-Context Learning (ICL)** was applied by directly providing the FAQ dataset (question–answer pairs) inside the model’s prompt.  
Instead of training or fine-tuning the model, the dataset was injected into the context, so whenever a user asks a question, the model uses the given context to generate the most relevant answer.  
This approach makes the model act like it has been “temporarily trained” on our FAQ dataset without modifying its internal parameters.

#### Limitations Compared to Fine-Tuning
- **Context size limit**: The entire FAQ dataset must fit into the model’s input prompt. Large datasets cannot be handled effectively.
- **No memory retention**: The model “forgets” the dataset once the session ends; unlike fine-tuning, it doesn’t permanently learn the knowledge.
- **Less accurate matching**: The model may sometimes generate approximate or unrelated answers if the question does not closely resemble the dataset.
- **Computation cost**: Each query requires passing the entire dataset into the prompt, which is inefficient compared to a fine-tuned model.

In summary, ICL is quick and flexible for small datasets, but fine-tuning is better for large, specialized FAQ systems.